## Data : livedoorニュースコーパス 9クラス分類

In [11]:
# ダウンロード
!wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2021-03-28 11:51:02--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M   322KB/s    in 33s     

2021-03-28 11:51:36 (263 KB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [12]:
import tarfile
import os
import json

In [13]:
## ファイルを解凍し、カテゴリー数と内容を確認
# 解凍
tar = tarfile.open("./data/ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)

['peachy', 'kaden-channel', 'README.txt', 'sports-watch', 'smax', 'topic-news', 'CHANGES.txt', 'it-life-hack', 'livedoor-homme', 'movie-enter', 'dokujo-tsushin']
カテゴリー数: 9
['peachy', 'kaden-channel', 'sports-watch', 'smax', 'topic-news', 'it-life-hack', 'livedoor-homme', 'movie-enter', 'dokujo-tsushin']


In [14]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある

0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



### 本文を取得する

In [15]:
# 前処理関数を定義
def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text

In [16]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)

In [17]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])

アメリカの女子たちに大人気のドラマ「ゴシップガール」のシーズン2、「ゴシップガール2」が、いよいよ4/15（木）21：00から、スーパー！ドラマTVにて放映されます。キュートなカップルだったセリーナとダン、格差カップルのネイトとヴァネッサ、そして恋よりプライド優先のブレアと女たらしのチャックという、まさかのカップルがそれぞれ破局？！というショッキングなエンディングで最終回を迎えたシーズン1。夏休みも終わりに近づき、彼らが再会するところからシーズン2は始まります。いったい、ブレアとチャックの恋の行方はどうなるのか？破局を迎えたセリーナとダンの復縁はあるのか？ 弱冠15歳のジェニーの“学園の女王”への野望は？家庭問題に翻弄されるネイトが頼る意外な人物とは？セリーナの母リリーとダンの父ルーファスの関係はいったいどうなるのか？ブレアやセリーナたちが高校生活最後の年を過ごす日々は、ファンの期待と予想をはるかに上回るドラマティックでスキャンダラスなストーリーが展開します。ニューヨークの本物のソーシャライトやデザイナーら、セレブが本人役で登場するところも、見逃せないポイント。話題のセレブやニューヨークのトレンドを語る上でも欠かせない、ホットなトピック満載の最新ＴＶシリーズ「ゴシップガール2」、ぜひチェックしてみて！■「ゴシップガール2」スーパー！ドラマTVにて、4/15（木）21：00 独占日本初放送スタート！二カ国語版・字幕版にて放送！毎週木曜21：00 ほか■関連リンク・ゴシップガール GOSSIPGIRL 公式サイト・スーパー！ドラマＴＶ
peachy


In [18]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()

(7376, 2)


,text,label
0,アメリカの女子たちに大人気のドラマ「ゴシップガール」のシーズン2、「ゴシップガール2」が、い...,peachy
1,この冬、スマフォであなたの生活を楽しくする「au×Hangame ドキドキ スマフォキャンペ...,peachy
2,味噌汁専門店の「美噌元」より、今話題のトマトを使ったフリーズドライ味噌汁「トマトとオクラ」が...,peachy
3,最近は「カメラ女子」なんて言葉もあるほど、女子の間でもデジタル一眼が大ブーム。今まではケータ...,peachy
4,“カタツムリ”で、何を連想しますか？日本人にとって、カタツムリはあまり馴染みのないものですが...,peachy


In [19]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

with open('./data/id2ctg.json', 'w') as f:
    json.dump(dic_id2cat, f)
with open('./data/ctg2id.json', 'w') as f:
    json.dump(dic_cat2id, f)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()

{0: 'peachy', 1: 'kaden-channel', 2: 'sports-watch', 3: 'smax', 4: 'topic-news', 5: 'it-life-hack', 6: 'livedoor-homme', 7: 'movie-enter', 8: 'dokujo-tsushin'}
{'peachy': 0, 'kaden-channel': 1, 'sports-watch': 2, 'smax': 3, 'topic-news': 4, 'it-life-hack': 5, 'livedoor-homme': 6, 'movie-enter': 7, 'dokujo-tsushin': 8}


,text,label_index
0,アメリカの女子たちに大人気のドラマ「ゴシップガール」のシーズン2、「ゴシップガール2」が、い...,0
1,この冬、スマフォであなたの生活を楽しくする「au×Hangame ドキドキ スマフォキャンペ...,0
2,味噌汁専門店の「美噌元」より、今話題のトマトを使ったフリーズドライ味噌汁「トマトとオクラ」が...,0
3,最近は「カメラ女子」なんて言葉もあるほど、女子の間でもデジタル一眼が大ブーム。今まではケータ...,0
4,“カタツムリ”で、何を連想しますか？日本人にとって、カタツムリはあまり馴染みのないものですが...,0


In [20]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,ドコモの「MEDIAS PP N-01D」は、高精細ディスプレイが魅力の防水スマートフォンだ...,5
1,世のサラリーマン＆OLが仕事帰りにゴルフの練習に行くことは、それなりに敷居が高い。ゴルフ道具...,6
2,ゼロから始めるスマートフォンKyoceraは8日（現地時間）、ニューオリンズ（アメリカ）にて...,3
3,「Windows上でどのように物理メモリが割り当てられているのか？」「メモリー上にどのくらい...,5
4,10月8日に韓国を訪問したオダギリジョーが「適当にサインをした」という疑惑を、今になって韓国...,4


### tsvファイルで保存する

In [21]:
# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./data/test.tsv", sep='\t', index=False)  # , header=None
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./data/train_eval.tsv", sep='\t', index=False)  # , header=None
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [22]:
df

,text,label_index
0,ドコモの「MEDIAS PP N-01D」は、高精細ディスプレイが魅力の防水スマートフォンだ...,5
1,世のサラリーマン＆OLが仕事帰りにゴルフの練習に行くことは、それなりに敷居が高い。ゴルフ道具...,6
2,ゼロから始めるスマートフォンKyoceraは8日（現地時間）、ニューオリンズ（アメリカ）にて...,3
3,「Windows上でどのように物理メモリが割り当てられているのか？」「メモリー上にどのくらい...,5
4,10月8日に韓国を訪問したオダギリジョーが「適当にサインをした」という疑惑を、今になって韓国...,4
...,...,...
7371,カモフラで遊び心を——。メンズファッションで定番のカモフラ（迷彩）柄を、ゴルフウェアで取り入...,6
7372,22日に放送された「FNS27時間テレビ」（フジテレビ系）の1コーナーで、「放送事故があった...,4
7373,「三大広告賞」と呼ばれているのはカンヌ国際広告祭、クリオ賞そして「One Show」だ。この...,1
7374,6月からさらに便利になるそうですよ！国内最大級の携帯電話や無線技術に関連したイベント「WIR...,3
